In [1]:
import pandas as pd

In [8]:
df_location = pd.read_csv("dataset/location.csv")

In [9]:
df_cases = pd.read_csv("dataset/individual.csv")

In [11]:
df_cases.age.count()

260490

In [17]:
df_cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 557364 entries, 0 to 557363
Data columns (total 10 columns):
age                       260490 non-null object
sex                       263630 non-null object
province                  550796 non-null object
country                   557340 non-null object
latitude                  557362 non-null float64
longitude                 557362 non-null float64
date_confirmation         556902 non-null object
additional_information    34395 non-null object
source                    348173 non-null object
outcome                   557364 non-null object
dtypes: float64(2), object(8)
memory usage: 42.5+ MB


In [19]:
df_cases.describe()

,latitude,longitude
count,557362.000000,557362.000000
mean,18.138385,27.285771
std,20.455801,67.577194
min,-54.808030,-159.727596
25%,11.042850,-58.473080
50%,19.036810,72.834830
75%,28.456000,77.209100
max,70.071800,174.740000
